In [33]:
import pandas as pd
from datetime import datetime

In [2]:
# read data
fact_order = pd.read_csv('data_raw/fact_order.csv',header = 0, nrows=100)
fact_order_variant_full = pd.read_csv('data_raw/fact_order_variant_full.csv',header = 0, nrows=100)
dim_customer = pd.read_csv('data_raw/dim_customer.csv',header = 0, nrows=100)
dim_store = pd.read_csv('data_raw/dim_store.csv',header = 0, nrows=100)
dim_target_sales = pd.read_csv('data_raw/dim_target_sales.csv',header = 0, nrows=100)
dim_variant = pd.read_csv('data_raw/dim_variant.csv',header = 0, nrows=100)

In [3]:
fact_order.head()

,order_id,order_number,created_at,source_name,store_id,user_id,email,customer_id,gateway,currency,...,utm_medium,utm_term,fulfillment_status,pos_date,sum_of_qty,count_of_variant,fullfillment_date,is_sync_salesforces,sync_salesforces_on,ref_salesforces_sync_history_id
0,1009717262,#100000,2017-05-19 14:30:45.027,pos,307222,1000204172,duckhanh@gmail.com,1003102514,tiền mặt,VND,...,NaN,NaN,fulfilled,2017-05-19 14:30:45.027,NaN,0,NaN,1,NaN,NaN
1,1009717652,#100001,2017-05-19 14:39:07.097,pos,307222,1000204172,duckhanh@gmail.com,1003102514,tiền mặt,VND,...,NaN,NaN,fulfilled,2017-05-19 14:39:07.097,NaN,0,NaN,1,NaN,NaN
2,1009717916,#100002,2017-05-19 14:45:15.787,web,295646,0,vuln.solaris@gmail.com,1003102663,Thanh toán khi giao hàng (COD),VND,...,NaN,NaN,notfulfilled,NaN,NaN,0,NaN,1,NaN,NaN
3,1009747265,#100003,2017-05-20 10:27:45.960,haravan_draft_order,295646,1000204172,guest@haravan.com,1003112749,Thanh toán khi giao hàng (COD),VND,...,NaN,NaN,fulfilled,2017-05-20 10:27:45.960,NaN,0,NaN,1,NaN,NaN
4,1009809428,#100004,2017-05-22 14:32:39.733,web,295646,0,guest+0123456789@haravan.com,1003129251,Thanh toán khi giao hàng (COD),VND,...,NaN,NaN,fulfilled,2017-05-22 14:32:39.733,NaN,0,NaN,1,NaN,NaN


For the simplicity of the project, we will only try to aggregate the sales data by date and store. Thus the column to use in fact_order will be `order_id; created_at; store_id; currency; fulfillment_status; pos_date`

In [9]:
# read data
usecols = ['order_id', 'created_at', 'store_id', 'currency', 'fulfillment_status', 'pos_date']

dtype_dict = {
    'store_id':'category',
    'currency':'category',
    'fulfillment_status':'category',
}

fact_order = pd.read_csv('data_raw/fact_order.csv',header = 0, usecols=usecols, dtype=dtype_dict)

In [10]:
# data frame info
fact_order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1510893 entries, 0 to 1510892
Data columns (total 6 columns):
 #   Column              Non-Null Count    Dtype   
---  ------              --------------    -----   
 0   order_id            1510893 non-null  int64   
 1   created_at          1510893 non-null  object  
 2   store_id            1510866 non-null  category
 3   currency            1510866 non-null  category
 4   fulfillment_status  1510866 non-null  category
 5   pos_date            1468380 non-null  object  
dtypes: category(3), int64(1), object(2)
memory usage: 38.9+ MB


In [29]:
for col in ['currency','fulfillment_status']:
    print(f"----------:\n{fact_order[col].value_counts()} ")

----------:
VND    1510866
Name: currency, dtype: int64 
----------:
fulfilled       1473734
notfulfilled      37132
Name: fulfillment_status, dtype: int64 


In [91]:
def processing(df, save_to_csv = False):
    df = df.query('fulfillment_status == "fulfilled"')     # only keep filfilled orders:
    df.created_at = df.created_at.map(lambda x:x[:10])  # get only date data
    df.created_at = pd.to_datetime(df['created_at'], format="%Y-%m-%d") 
    df = df.groupby(['created_at','store_id']).sum().reset_index()
    df.order_id.fillna(0,inplace=True)
    if save_to_csv:
        df.to_csv("agg_sales_store_daily.csv")
    return df

In [92]:
df = fact_order.copy()
processing(df, save_to_csv =True)

C:\Users\Admin\anaconda3\envs\DataMining_Lab\lib\site-packages\pandas\core\generic.py:5159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,created_at,store_id,order_id
0,2017-05-19,295646,0.000000e+00
1,2017-05-19,307222,2.019435e+09
2,2017-05-19,307244,0.000000e+00
3,2017-05-19,307248,0.000000e+00
4,2017-05-19,320264,0.000000e+00
...,...,...,...
128565,2021-08-24,947306,0.000000e+00
128566,2021-08-24,947302,0.000000e+00
128567,2021-08-24,947298,0.000000e+00
128568,2021-08-24,1005758,6.115058e+09
